### [전처리 단계 설계]
1. none
2. 기호제거
3. 한자변환 + 기호제거
4. kospacing
5. kospacing + 기호제거
6. kospacing + 한자변환 + 기호제거

In [1]:
import pandas as pd

In [2]:
from soynlp.noun import NewsNounExtractor

## AIMS

In [3]:
%time AIMS = pd.read_excel('data/HKMC_MEL_MST_LIST.xlsx')

Wall time: 4min 13s


In [4]:
df_aims = AIMS[['CAU_DESC','REQ_DESC','RSL_DESC']].drop_duplicates()
len(df_aims)

137355

In [5]:
df = pd.DataFrame()

In [6]:
df['content'] = df_aims['CAU_DESC'] + '. ' + df_aims['REQ_DESC'] + '. ' + df_aims['RSL_DESC'] + '.'

In [7]:
df = df.dropna()

In [8]:
len(df)

108865

In [9]:
df.to_csv('result/org.csv', encoding='utf-8', index=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108865 entries, 0 to 172300
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  108865 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


1. none 

In [10]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df['content'].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (847145, 518475, 369889)
predicting noun score was done                                        
before postprocessing 191085
_noun_scores_ 21081
checking hardrules ... done0 / 21081+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 15658
extracted 3419 compounds from eojeolss ... 38000 / 38792

In [11]:
type(nouns)

dict

In [12]:
nouns_n = list(nouns.keys())

2. 기호제거

In [13]:
df['content_rm_ss'] = df['content'].str.replace(pat=r'[^가-힣A-Za-z0-9]', repl= r' ', regex=True)  # replace all special symbols to space
df['content_rm_ss'] = df['content_rm_ss'].str.replace(pat=r'[\n|\r|\t]', repl= r' ', regex=True)
df['content_rm_ss'] = df['content_rm_ss'].str.replace(pat=r'[\s\s+]', repl= r' ', regex=True)  # replace multiple spaces with a single space

In [14]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df['content_rm_ss'].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (330237, 212702, 182306)
predicting noun score was done                                        
before postprocessing 106211
_noun_scores_ 18349
checking hardrules ... done0 / 18349+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 14060
extracted 5702 compounds from eojeolss ... 31000 / 31732

In [15]:
nouns_rm_ss = list(nouns.keys())

3. 한자변환 + 기호제거

In [16]:
import hanja

In [22]:
df['content_rm_hanja'] = df['content'].apply(lambda x : hanja.translate(x, 'substitution'))

In [23]:
df['content_rm_hanja_ss'] = df['content_rm_hanja'].str.replace(pat=r'[^가-힣A-Za-z0-9]', repl= r' ', regex=True)  # replace all special symbols to space
df['content_rm_hanja_ss'] = df['content_rm_hanja_ss'].str.replace(pat=r'[\n+]', repl= r' ', regex=True)
df['content_rm_hanja_ss'] = df['content_rm_hanja_ss'].str.replace(pat=r'[\s\s+]', repl= r' ', regex=True)  # replace multiple spaces with a single space

In [21]:
df.to_csv('result/check.csv', encoding='utf-8', index=False)

In [24]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df['content_rm_hanja_ss'].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (335269, 216005, 184691)
predicting noun score was done                                        
before postprocessing 106727
_noun_scores_ 18402
checking hardrules ... done0 / 18402+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 14084
extracted 6102 compounds from eojeolss ... 31000 / 31829

In [25]:
content_rm_hanja_ss = list(nouns.keys())

4. kospacing

In [26]:
# df_kospacing = pd.read_('data/kospacing.dap', encoding='utf-8')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 11, saw 4


In [30]:
with open('data/kospacing.dap','r', encoding='utf-8') as f:
    next(f) # skip first row
    df_kospacing = pd.DataFrame(l for l in f)

In [31]:
df_kospacing.head()

,0
0,리어 트랜스버스트림 끼움 시 들뜸 및 갭 발생. 걸림턱 수정. 리어 트랜스버스 트림...
1,클러스터 페시아 끼움 어려움. 클립 걸림턱 높이 낮춤. 걸림턱 수정 반영 예정.\n
2,"리어 시트 슬라이딩 시 아크릴 커버 매트에 의 해 커넥터 파손 발생, 러기지 사이드..."
3,"센터페시아 판넬 끼움 어려움 원인 : 변속 레버 측 끼움 클립 매칭 어렵고, 클립 ..."
4,크래시 패드 로워 판넬 장착 어려움 원인 : 가이드핀(상어지느러미 모양) 끼움 어려...


In [32]:
len(df_kospacing)

108864

In [35]:
df_kospacing[0].tolist()

['리어 트랜스버스트림 끼움 시 들뜸 및 갭 발생. 걸림턱 수정. 리어 트랜스버스 트림 상부 클립 보강 구조 커팅 하단 갭은 문제 없음.\n',
 '클러스터 페시아 끼움 어려움. 클립 걸림턱 높이 낮춤. 걸림턱 수정 반영 예정.\n',
 '리어 시트 슬라이딩 시 아크릴 커버 매트에 의 해 커넥터 파손 발생, 러기지 사이드 트림 스크류 체결부에 걸림. 1. 커버 매트 소재 변경(아크릴→pp) 2. 매트 모서리 곡면 형상으로 변경 . 매트 소재 변경 및 라운딩 반영 예정.\n',
 '센터페시아 판넬 끼움 어려움 원인 : 변속 레버 측 끼움 클립 매칭 어렵고, 클립 변형 발생. 변속 레버 측 고정 클립 삭제 혹은 가이드 구조로 변경 (합동조립시 해당 클립 컷팅 삭제 후 작업 후 레버 조작 시 유동 없음). 2차 합동조립 재확인 후, 가이드 핀 구조 변경 혹은 핀 삭제.\n',
 '크래시 패드 로워 판넬 장착 어려움 원인 : 가이드핀(상어지느러미 모양) 끼움 어려움. 가이드 핀 길이 증대(10mm). 길이 증대 반영 예정.\n',
 '코트 후크 브라켓 체결 안됨 (차체 측 스토퍼 홀 확공, 브라켓 측 체결홀 확공 리 워크 후 작업). 차체 측 스토퍼 홀 증대, 브라켓 측 볼트 체결홀 증대. 부품 실물 재확인.\n',
 '센터필라어 퍼 트림 하단 클립 끼움 어려움 원인 : 상단 후크 조립 어려움. 센터필라어 퍼 트림 상부 후크 커팅. [개선사양 조립성 TO 예정] 1. 문제점 : 센터필라트림 장착 어려움 2.원인 : 트림 상단 후크 미조립 3. 개선안 : 후크 걸림 부위 커팅 (커팅량 TO 확인 예정).\n',
 '코크 후크 체결 시 회전 (가이드 핀 역할 불능). 가이드 핀 두께 증대 요청. 코트 후크 브라켓 측 가이드 핀 홀 삽입 위치 변경하여 시계 방향 회전 흡수할 수 있는 구조 적용 예정.\n',
 '히터 컨트롤러 체결 시 CPAD 센터 로워 가이드홀 틀 어짐. 차주 C/PAD 업체에서 재확인 (스마트 디엔에스, 10/29~30). 부품 디멘젼 재확인.\n',


In [36]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df_kospacing[0].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (582021, 347122, 280447)
predicting noun score was done                                        
before postprocessing 146612
_noun_scores_ 14146
checking hardrules ... done0 / 14146+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 10414
extracted 1960 compounds from eojeolss ... 36000 / 36536

In [37]:
nouns_kospacing = list(nouns.keys())

5. kospacing + 기호제거

In [38]:
df_kospacing[1] = df_kospacing[0].str.replace(pat=r'[^가-힣A-Za-z0-9]', repl= r' ', regex=True)  # replace all special symbols to space
df_kospacing[1] = df_kospacing[1].str.replace(pat=r'[\n+]', repl= r' ', regex=True)
df_kospacing[1] = df_kospacing[1].str.replace(pat=r'[\s\s+]', repl= r' ', regex=True)  # replace multiple spaces with a single space

In [39]:
df_kospacing.head()

,0,1
0,리어 트랜스버스트림 끼움 시 들뜸 및 갭 발생. 걸림턱 수정. 리어 트랜스버스 트림...,리어 트랜스버스트림 끼움 시 들뜸 및 갭 발생 걸림턱 수정 리어 트랜스버스 트림...
1,클러스터 페시아 끼움 어려움. 클립 걸림턱 높이 낮춤. 걸림턱 수정 반영 예정.\n,클러스터 페시아 끼움 어려움 클립 걸림턱 높이 낮춤 걸림턱 수정 반영 예정
2,"리어 시트 슬라이딩 시 아크릴 커버 매트에 의 해 커넥터 파손 발생, 러기지 사이드...",리어 시트 슬라이딩 시 아크릴 커버 매트에 의 해 커넥터 파손 발생 러기지 사이드...
3,"센터페시아 판넬 끼움 어려움 원인 : 변속 레버 측 끼움 클립 매칭 어렵고, 클립 ...",센터페시아 판넬 끼움 어려움 원인 변속 레버 측 끼움 클립 매칭 어렵고 클립 ...
4,크래시 패드 로워 판넬 장착 어려움 원인 : 가이드핀(상어지느러미 모양) 끼움 어려...,크래시 패드 로워 판넬 장착 어려움 원인 가이드핀 상어지느러미 모양 끼움 어려...


In [40]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df_kospacing[1].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (187003, 108833, 118902)
predicting noun score was done                                        
before postprocessing 71155
_noun_scores_ 11135
checking hardrules ... done0 / 11135+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 8405
extracted 2495 compounds from eojeolss ... 26000 / 26168

In [41]:
nouns_kospacing_rm_ss = list(nouns.keys())

6. kospacing + 한자변환 + 기호제거

In [42]:
df_kospacing[2] = df_kospacing[0].apply(lambda x : hanja.translate(x, 'substitution'))

In [43]:
df_kospacing[3] = df_kospacing[2].str.replace(pat=r'[^가-힣A-Za-z0-9]', repl= r' ', regex=True)  # replace all special symbols to space
df_kospacing[3] = df_kospacing[3].str.replace(pat=r'[\n+]', repl= r' ', regex=True)
df_kospacing[3] = df_kospacing[3].str.replace(pat=r'[\s\s+]', repl= r' ', regex=True)  # replace multiple spaces with a single space

In [44]:
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(df_kospacing[3].tolist())

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/H2009134/AppData/Local/Continuum/anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (191713, 111816, 121125)
predicting noun score was done                                        
before postprocessing 71675
_noun_scores_ 11188
checking hardrules ... done0 / 11188+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 8432
extracted 3273 compounds from eojeolss ... 26000 / 26315

In [45]:
nouns_kospacing_rm_hanja_ss = list(nouns.keys())

In [62]:
nouns_n.sort()
nouns_rm_ss.sort()
content_rm_hanja_ss.sort()
nouns_kospacing.sort()
nouns_kospacing_rm_ss.sort()
nouns_kospacing_rm_hanja_ss.sort()

In [63]:
print(len(nouns_n))
print(len(nouns_rm_ss))
print(len(content_rm_hanja_ss))
print(len(nouns_kospacing))
print(len(nouns_kospacing_rm_ss))
print(len(nouns_kospacing_rm_hanja_ss))

19077
19762
20186
12374
10900
11705


In [67]:
df_comp_keyword = pd.DataFrame()

In [68]:
df_comp_keyword['nouns_n'] = nouns_n + (['nan'] * (len(content_rm_hanja_ss)-len(nouns_n)))
df_comp_keyword['nouns_rm_ss'] = nouns_rm_ss + (['nan'] * (len(content_rm_hanja_ss)-len(nouns_rm_ss)))
df_comp_keyword['nouns_rm_hanja_ss'] = content_rm_hanja_ss
df_comp_keyword['nouns_kospacing'] = nouns_kospacing + (['nan'] * (len(content_rm_hanja_ss)-len(nouns_kospacing)))
df_comp_keyword['nouns_kospacing_rm_ss'] = nouns_kospacing_rm_ss + (['nan'] * (len(content_rm_hanja_ss)-len(nouns_kospacing_rm_ss)))
df_comp_keyword['nouns_kospacing_rm_hanja_ss'] = nouns_kospacing_rm_hanja_ss + (['nan'] * (len(content_rm_hanja_ss)-len(nouns_kospacing_rm_hanja_ss)))

In [65]:
df_comp_keyword.head()

,content_rm_hanja_ss,nouns_n,nouns_rm_ss,nouns_kospacing,nouns_kospacing_rm_ss,nouns_kospacing_rm_hanja_ss
0,000원,"""-T방향""",000원,"""-T방향""",0000,0000
1,002,"""0""터치",002,"""0""터치",000원,000원
2,01020,"""L""면",01020,"""L""면",002,002
3,0118협의결,"""L""방향",0118협의결,"""L""방향",003번,003번
4,017,"""L""자",017,"""L""자",006,006


In [69]:
df_comp_keyword.to_csv('result/df_comp_keyword.csv', encoding='utf-8')

### soyspacing, 다른 토크나이저 적용해서 비교해보기!

- LRNounExtractor
- NewsNounExtractor
- LRNounExtractor_v2
- WordExtractor

- LTokenizer
- MaxScoreTokenizer
- RegexTokenizer
- NounLMatchTokenizer